### IMPORTANDO AS BIBLIOTECAS

In [1]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle

### LEITURA DA BASE DE DADOS

In [22]:
df = pd.read_parquet("../data/processed/sim_2006_2017_balanced.parquet") 

### VISUALIZAÇÃO DA BASE DE DADOS

In [23]:
df

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,MASCULINO,FEMININO,SUICIDIO
0,-0.862039,-0.611299,-0.690103,-1.796410,-0.349766,-0.820889,False,True,0
1,-0.862039,1.180033,-0.711939,-1.796410,-0.350851,-0.820889,True,False,0
2,-0.739565,-0.867204,-0.801467,0.549301,-0.350509,-0.820889,True,False,0
3,-0.862039,-0.355395,-0.694470,0.549301,-0.350380,-0.820889,False,True,0
4,-0.047954,-0.611299,-0.694470,0.549301,-0.349766,0.076121,False,True,0
...,...,...,...,...,...,...,...,...,...
272141,0.882637,-0.014159,0.458619,-0.623555,2.317550,1.574331,False,True,1
272142,0.882637,-0.014159,1.482439,-0.623555,-0.350724,1.574331,False,True,1
272143,1.423335,0.434921,1.041437,-0.623555,2.317550,-0.028007,True,False,1
272144,1.006996,0.434921,1.015247,-0.623555,-0.350724,1.574331,True,False,1


### BALANCEAMENTO DA BASE DE DADOS EM CASOS DE SUICÍDIO

In [24]:
suicidio_0 = df[df['SUICIDIO'] == 0]
suicidio_1 = df[df['SUICIDIO'] == 1]

suicidio_0_sample = suicidio_0.sample(n=len(suicidio_1), replace=False)

balanced_df = pd.concat([suicidio_0_sample, suicidio_1], ignore_index=True).reset_index(drop=True)

#### Definindo X e Y de acordo com a base de dados balanceada

In [25]:
X = balanced_df[['ESC', 'ESTCIV', 'IDADE' ,	'NATURAL' ,	'OCUP',	'RACACOR'	,'MASCULINO',	'FEMININO'	]].values
Y = balanced_df['SUICIDIO'].values

#### Divisão entre teste e treino 

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=666)

### TREINANDO O MODELO
Modelo escolhido: Centróides.
Como nossa base de dados já apresenta os dados de suicídio, não foi necessária a clusterização. Portanto, os centróides de clusters podem ser utilizados para identificar os pontos centrais. Cada centróide representa os aspectos gerais de uma determinada classe.

In [27]:
modelo_prototipos = NearestCentroid()

In [28]:
modelo_prototipos.fit(X_train, y_train)

NearestCentroid()

### PREVISÃO DO MODELO TREINADO E ACURÁCIA

In [29]:
y_pred = modelo_prototipos.predict(X_test)

In [30]:
relatorio = classification_report(y_test, y_pred)
matriz = confusion_matrix(y_test, y_pred)

In [31]:
print("Relatório de Classificação:\n", relatorio)
print("Matriz de Confusão:\n", matriz)

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.85      0.99      0.92     27062
           1       0.99      0.83      0.90     27368

    accuracy                           0.91     54430
   macro avg       0.92      0.91      0.91     54430
weighted avg       0.92      0.91      0.91     54430

Matriz de Confusão:
 [[26916   146]
 [ 4775 22593]]


### VISUALIZAÇÃO DOS DADOS

In [32]:
import seaborn as sns

# Plotar gráficos de distribuição das features em relação à variável alvo
#sns.pairplot(balanced_df, hue='SUICIDIO', vars=['ESC', 'ESTCIV', 'IDADE', 'NATURAL', 'OCUP', 'RACACOR', 'MASCULINO', 'FEMININO'])

In [33]:
geovanna = pd.DataFrame({'ESC': 5,    'ESTCIV': 1,    'IDADE': 22,    'NATURAL': 53,    'OCUP': -1,    'RACACOR': 4,    'MASCULINO': False,    'FEMININO': True}, index=[0])

arthur = pd.DataFrame({'ESC': 5,    'ESTCIV': 1,    'IDADE': 21,    'NATURAL': 52,    'OCUP': -1,    'RACACOR': 1,    'MASCULINO': True,    'FEMININO': False}, index=[0])



doug = pd.DataFrame({'ESC': 4,    'ESTCIV': 2,    'IDADE': 34,    'NATURAL': 29,    'OCUP': 5132,    'RACACOR': 4,    'MASCULINO': True,    'FEMININO': False}, index=[0])



cozinheiro = pd.DataFrame({'ESC': 4,    'ESTCIV': 2,    'IDADE': 34,    'NATURAL': 29,    'OCUP': 5132,    'RACACOR': 4,    'MASCULINO': True,    'FEMININO': False}, index=[0])


In [71]:
teste = pd.concat((geovanna, arthur, doug, cozinheiro), ignore_index=True)

In [68]:
with open("../models/stdscaler.pkl", "rb") as f:
    std=pickle.load((f))

In [72]:
columns = ['ESC', 'ESTCIV', 'IDADE', 'OCUP', 'RACACOR', 'NATURAL']
teste[columns] = std.transform(teste[columns])

x = teste[['ESC', 'ESTCIV', 'IDADE', 'OCUP', 'RACACOR', 'NATURAL', 'MASCULINO', 'FEMININO']].values

In [73]:
y_pred = modelo_prototipos.predict(x)
y_pred


array([1, 1, 1, 1])